<a href="https://colab.research.google.com/github/s-Arathy/AI_Editing_Assistant/blob/main/Copy_of_ai_film_editing_assistant_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Film Editing Assistant (Colab-Ready)

This notebook suggests intelligent cut points and transitions by analyzing video scenes, dialogue, and emotional shifts.

- Input: a video file (mp4/mov). We'll extract scenes, transcribe audio, estimate emotions, and fuse signals to propose cuts.
- Output: CSV/EDL of suggested cut timestamps, an interactive timeline, and an optional preview render.

Quickstart:
1) Run Setup below. 2) Provide `video_path` or download the sample. 3) Run cells in order.

Sample video (CC-BY): Blender Foundation's "Tears of Steel" trailer.



In [ ]:
# Setup: dependencies (runs on Colab or local)
import sys, subprocess
print(sys.version)

# Use IPython magics so this works in notebooks
%pip install -q --upgrade pip wheel setuptools
# Ensure a modern IPython and install jedi to satisfy dependency
%pip install -q "ipython>=8.12" jedi>=0.18.0
%pip install -q opencv-python scenedetect moviepy librosa torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers==4.43.3 datasets==2.20.0 \
  openai-whisper==20231117 \
  faster-whisper==1.0.3 \
  jiwer==3.0.5 \
  pandas plotly==5.22.0 ipywidgets==8.1.3 \
  pyloudnorm==0.1.1

# ffmpeg check
!ffmpeg -version



3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-whisper 20231117 requires triton<3,>=2.0.0, but you have triton 3.4.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.23.0+cu126 requires torch==2.8.0, but you have torch 2.3.1 which is incompatible.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.3.1 which is incompatible.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gp

In [ ]:
import os, sys, json, math, itertools, tempfile, shutil, subprocess
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dataclasses import dataclass

SAMPLE_URLS = [
    # Public sample videos
    "https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4",
    "https://media.w3.org/2010/05/sintel/trailer.mp4",
]

@dataclass
class Config:
    video_path: str = ""  # set your path or leave empty to download sample
    work_dir: str = "work"
    min_scene_len_s: float = 1.5
    whisper_model: str = "medium"  # or "small" for speed
    emotion_model: str = "j-hartmann/emotion-english-distilroberta-base"

cfg = Config()
Path(cfg.work_dir).mkdir(exist_ok=True)

print("Set `cfg.video_path` to your file path if not using the sample.")



Set `cfg.video_path` to your file path if not using the sample.


In [ ]:
# Download or mount a video
from urllib.request import Request, urlopen

def download_with_headers(url: str, dest: Path):
    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urlopen(req) as r, open(dest, 'wb') as f:
        shutil.copyfileobj(r, f)

def ensure_video(path: str) -> str:
    if path and Path(path).exists():
        return path
    out = Path(cfg.work_dir) / "sample.mp4"
    if not out.exists():
        print("Downloading sample video...")
        for url in SAMPLE_URLS:
            try:
                download_with_headers(url, out)
                break
            except Exception as e:
                print("Failed:", url, e)
        if not out.exists():
            raise RuntimeError("Could not download sample video. Please set cfg.video_path to a local file.")

    return str(out)

video_path = ensure_video(cfg.video_path)
video_path



'work/sample.mp4'

In [ ]:
# Scene detection with PySceneDetect
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector

video = open_video(video_path)
scene_manager = SceneManager()
scene_manager.add_detector(ContentDetector(threshold=27.0, min_scene_len=int(cfg.min_scene_len_s*video.base_timecode.framerate)))
scene_manager.detect_scenes(video)
scene_list = scene_manager.get_scene_list()

scenes = []
for i, (start, end) in enumerate(scene_list):
    scenes.append({
        'scene': i,
        'start_s': start.get_seconds(),
        'end_s': end.get_seconds(),
        'dur_s': end.get_seconds() - start.get_seconds(),
    })
scenes_df = pd.DataFrame(scenes)
print(len(scenes_df), "scenes detected")
scenes_df.head()



/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
INFO:pyscenedetect:Detecting scenes...


114 scenes detected


,scene,start_s,end_s,dur_s
0,0,0.000000,11.916667,11.916667
1,1,11.916667,15.791667,3.875000
2,2,15.791667,23.083333,7.291667
3,3,23.083333,47.750000,24.666667
4,4,47.750000,56.125000,8.375000


In [ ]:
# Extract representative keyframes for the first few scenes (preview)
import cv2

cap = cv2.VideoCapture(video_path)
keyframes_dir = Path(cfg.work_dir) / "keyframes"
keyframes_dir.mkdir(exist_ok=True)

preview = []
for _, row in scenes_df.head(8).iterrows():
    t = (row['start_s'] + row['end_s'])/2
    cap.set(cv2.CAP_PROP_POS_MSEC, t*1000)
    ok, frame = cap.read()
    if not ok:
        continue
    out_path = keyframes_dir / f"scene_{int(row['scene']):03d}.jpg"
    cv2.imwrite(str(out_path), frame)
    preview.append({'scene': int(row['scene']), 'keyframe_path': str(out_path)})
cap.release()

pd.DataFrame(preview)



,scene,keyframe_path
0,0,work/keyframes/scene_000.jpg
1,1,work/keyframes/scene_001.jpg
2,2,work/keyframes/scene_002.jpg
3,3,work/keyframes/scene_003.jpg
4,4,work/keyframes/scene_004.jpg
5,5,work/keyframes/scene_005.jpg
6,6,work/keyframes/scene_006.jpg
7,7,work/keyframes/scene_007.jpg


In [ ]:
# Speech-to-text using Whisper (faster-whisper for speed)
from faster_whisper import WhisperModel
import subprocess, json, librosa

# Extract audio to wav
wav_path = str(Path(cfg.work_dir)/"audio.wav")
if not Path(wav_path).exists():
    _ = subprocess.run([
        'ffmpeg','-y','-i',video_path,'-vn','-ac','1','-ar','16000','-f','wav',wav_path
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Choose device/model; use smaller model on CPU for speed
device = "cuda" if shutil.which('nvidia-smi') else "cpu"
model_size = cfg.whisper_model if device == "cuda" else "small"
compute_type = "float16" if device == "cuda" else "int8"
model = WhisperModel(model_size, device=device, compute_type=compute_type)

# If on CPU and long audio, cap to 3 minutes to prevent stalls (adjust as needed)
limited_wav = wav_path
if device == "cpu":
    try:
        dur = librosa.get_duration(path=wav_path)
        if dur and dur > 180:
            limited_wav = str(Path(cfg.work_dir)/"audio_3min.wav")
            _ = subprocess.run([
                'ffmpeg','-y','-i', wav_path, '-t', '180', '-acodec', 'copy', limited_wav
            ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print("CPU mode: capped transcription to first 180s for speed.")
    except Exception:
        pass

# Chunked, non-VAD transcription (more stable) with light settings
transcripts = []
seg_iter, info = model.transcribe(
    limited_wav,
    language="en",
    task="transcribe",
    beam_size=1,
    best_of=1,
    temperature=0.0,
    condition_on_previous_text=False,
    vad_filter=False,
    chunk_length=15,
)
for seg in seg_iter:
    transcripts.append({'start_s': seg.start, 'end_s': seg.end, 'text': seg.text.strip()})

trans_df = pd.DataFrame(transcripts)
print(f"Transcribed {len(trans_df)} segments on {device} with model '{model_size}'")
trans_df.head()



tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

CPU mode: capped transcription to first 180s for speed.
Transcribed 23 segments on cpu with model 'small'


,start_s,end_s,text
0,0.0,9.0,Music
1,9.0,14.0,I'm not sure what to do with this.
2,14.0,19.0,I'm just going to go back to the castle.
3,19.0,24.0,I'm going to go back to the castle.
4,24.0,29.0,I'm going to go back to the castle.


In [ ]:
# Emotion analysis with transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

id2label = {
  0: 'anger', 1: 'disgust', 2: 'fear', 3: 'joy', 4: 'neutral', 5: 'sadness', 6: 'surprise'
}

model_name = cfg.emotion_model
tokenizer = AutoTokenizer.from_pretrained(model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(model_name)
emotion_model.eval()

def score_emotions(texts, batch_size=16):
    scores = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            toks = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            logits = emotion_model(**toks).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
            scores.extend(probs)
    return np.array(scores)

emo_scores = score_emotions(trans_df['text'].tolist())
emo_cols = [f"emo_{i}" for i in range(emo_scores.shape[1])]
emotions_df = pd.DataFrame(emo_scores, columns=emo_cols)
trans_emotions_df = pd.concat([trans_df.reset_index(drop=True), emotions_df], axis=1)
trans_emotions_df.head()



tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

,start_s,end_s,text,emo_0,emo_1,emo_2,emo_3,emo_4,emo_5,emo_6
0,0.0,9.0,Music,0.008787,0.023284,0.010223,0.030392,0.797876,0.092759,0.036678
1,9.0,14.0,I'm not sure what to do with this.,0.040721,0.078298,0.090197,0.003125,0.487461,0.051841,0.248358
2,14.0,19.0,I'm just going to go back to the castle.,0.044091,0.090943,0.049565,0.047407,0.732262,0.029429,0.006302
3,19.0,24.0,I'm going to go back to the castle.,0.054960,0.060638,0.398440,0.121814,0.260593,0.093540,0.010015
4,24.0,29.0,I'm going to go back to the castle.,0.054960,0.060638,0.398440,0.121814,0.260593,0.093540,0.010015


In [ ]:
# Fuse signals to suggest cuts

def moving_std(x, w=5):
    x = np.asarray(x)
    if len(x) < w:
        return np.zeros_like(x)
    pad = w//2
    xpad = np.pad(x, (pad, pad), mode='edge')
    out = []
    for i in range(len(x)):
        win = xpad[i:i+w]
        out.append(np.std(win))
    return np.array(out)

# Derive an "emotional change" score from max-prob label
emo_max = trans_emotions_df[[c for c in trans_emotions_df.columns if c.startswith('emo_')]].values.max(axis=1)
emo_var = moving_std(emo_max, w=7)

# Dialogue activity signal (segment density per second)
seg_centers = (trans_emotions_df['start_s'] + trans_emotions_df['end_s'])/2

# Candidate cut points: scene boundaries + strong emotion change peaks
scene_bounds = np.sort(np.r_[scenes_df['start_s'].values, scenes_df['end_s'].values])

# Peaks via simple thresholding
thr = np.percentile(emo_var, 85)
emo_indices = np.where(emo_var >= thr)[0]
candidate_times = set(float(np.round(seg_centers.iloc[i], 2)) for i in emo_indices)
for t in scene_bounds:
    candidate_times.add(float(np.round(t, 2)))

candidates = sorted(candidate_times)

# Remove too-close candidates (< 1.0s)
filtered = []
for t in candidates:
    if not filtered or t - filtered[-1] >= 1.0:
        filtered.append(t)

suggestions_df = pd.DataFrame({'time_s': filtered})
print(len(suggestions_df), 'candidate cut points')
suggestions_df.head(20)



117 candidate cut points


,time_s
0,0.00
1,4.50
2,11.50
3,15.79
4,21.50
5,23.08
6,47.75
7,56.12
8,60.71
9,63.83


In [ ]:
# Interactive timeline visualization
import plotly.express as px

fig = go.Figure()

# Add scene spans
for _, r in scenes_df.iterrows():
    fig.add_vrect(x0=r['start_s'], x1=r['end_s'], fillcolor="#1f77b4", opacity=0.08, line_width=0)

# Emotion variability line
fig.add_trace(go.Scatter(x=seg_centers, y=emo_var, name='emotion_change', line=dict(color='#e45756')))

# Suggested cuts
fig.add_trace(go.Scatter(x=suggestions_df['time_s'], y=[max(emo_var)*1.02]*len(suggestions_df),
                         mode='markers', name='suggested cuts', marker=dict(color='#2ca02c', size=8)))

fig.update_layout(height=350, xaxis_title='time (s)', yaxis_title='score', title='Timeline: scenes, emotion change, suggested cuts')
fig.show()



In [ ]:
# Export EDL/CSV and optional preview render
from moviepy.editor import VideoFileClip, concatenate_videoclips

cuts_csv = Path(cfg.work_dir)/"suggested_cuts.csv"
suggestions_df.to_csv(cuts_csv, index=False)
print("Saved:", cuts_csv)

# Basic rough-cut preview: split at suggested cuts and keep 2s around each
make_preview = True
preview_path = Path(cfg.work_dir)/"preview.mp4"
if make_preview:
    clip = VideoFileClip(video_path)
    parts = []
    for t in suggestions_df['time_s']:
        start = max(0, t-1.0)
        end = min(clip.duration, t+1.0)
        if end > start:
            parts.append(clip.subclip(start, end))
    if parts:
        out = concatenate_videoclips(parts, method='compose')
        out.write_videofile(str(preview_path), codec='libx264', audio_codec='aac', verbose=False, logger=None)
        print("Preview rendered:", preview_path)
    else:
        print("No parts to render")



Saved: work/suggested_cuts.csv


Preview rendered: work/preview.mp4
